In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

data_path = r'C:\Users\Sureka Siriwardana\Desktop\FYP\DATA\output.csv'
df = pd.read_csv(data_path)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])


In [2]:
X = df[['Differential Value']].values
y = df['Label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y)


In [3]:
def create_model(input_shape):
    model = Sequential([
        LSTM(50, input_shape=input_shape),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
## For K Fold Cross validation 
k = 5 
kf = KFold(n_splits=k, shuffle=True, random_state=42)


In [4]:
fold_no = 1
for train_index, test_index in kf.split(X_seq):
    X_train, X_test = X_seq[train_index], X_seq[test_index]
    y_train, y_test = y_seq[train_index], y_seq[test_index]

    model = create_model((X_train.shape[1], X_train.shape[2]))
    print(f'Training for fold {fold_no}...')
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    predictions = (model.predict(X_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)

    print(f'Fold {fold_no} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')
    fold_no += 1


Training for fold 1...
Epoch 1/20
1175/1175 [==============================] - 15s 10ms/step - loss: 0.1228 - accuracy: 0.9752 - val_loss: 0.1044 - val_accuracy: 0.9763
Epoch 2/20
1175/1175 [==============================] - 12s 10ms/step - loss: 0.1009 - accuracy: 0.9760 - val_loss: 0.0978 - val_accuracy: 0.9781
Epoch 3/20
1175/1175 [==============================] - 11s 9ms/step - loss: 0.0947 - accuracy: 0.9758 - val_loss: 0.0935 - val_accuracy: 0.9773
Epoch 4/20
1175/1175 [==============================] - 12s 10ms/step - loss: 0.0927 - accuracy: 0.9763 - val_loss: 0.0905 - val_accuracy: 0.9771
Epoch 5/20
1175/1175 [==============================] - 12s 10ms/step - loss: 0.0913 - accuracy: 0.9761 - val_loss: 0.0876 - val_accuracy: 0.9768
Epoch 6/20
1175/1175 [==============================] - 12s 10ms/step - loss: 0.0902 - accuracy: 0.9763 - val_loss: 0.0863 - val_accuracy: 0.9771
Epoch 7/20
1175/1175 [==============================] - 12s 10ms/step - loss: 0.0902 - accuracy: 0.976

In [5]:
# Save the LSTM model
model.save('lstm_model.h5')


In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

model = load_model('lstm_model.h5')

def preprocess_data(data, scaler):

    data = np.array(data).reshape(-1, 1)
    data_scaled = scaler.transform(data)
    time_steps = 10
    samples = len(data_scaled)
    Xs = []
    for i in range(samples - time_steps + 1):
        Xs.append(data_scaled[i:(i + time_steps)])
    return np.array(Xs)

def predict(data):
    
    scaler = StandardScaler().fit(data.reshape(-1, 1))  
    
    X = preprocess_data(data, scaler)
    predictions = model.predict(X)
    
    predictions = (predictions > 0.5).astype(int)
    return predictions

def get_inference():
    try:
       
        data_points = input("Enter the differential values, separated by commas (e.g., -3, 1, 0): ")
        data_points = [float(value.strip()) for value in data_points.split(',')]
        if len(data_points) < 10:
            raise ValueError("Please enter at least 10 data points for the sequence.")

        
        predictions = predict(np.array(data_points))
        print("Predictions:", predictions)
    except Exception as e:
        print(f"An error occurred: {e}")

get_inference()


1/1 [==============================] - 0s 454ms/step
Predictions: [[1]]
